In [22]:
# importing the lib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy
import time
import sys
from selenium.webdriver.chrome.service import service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re
import numpy as np

In [123]:
def kayak(doj,souce_airport,destination_airport):
    url="https://www.ca.kayak.com/flights/" +str(source_airport) +"-" +str(destination_airport)+ "/"+str(doj)+ "?sort=bestflight_a"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.maximize_window()
    driver.get(url)
    time.sleep(15)
    def click_more():
    
        for val in range(0,50):
            try:
                driver.find_element_by_xpath("//a[@class='moreButton']").click()
                time.sleep(2)
                val = val +1

            except:
                pass
    click_more()
    
    # depart-time base-time
    departure_time=[]
    departure_time= driver.find_elements_by_xpath("//span[@class='depart-time base-time']")

    arrival_time=[]
    arrival_time= driver.find_elements_by_xpath("//span[@class='arrival-time base-time']")

    duration =[]
    duration  = driver.find_elements_by_xpath("//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")

    no_stop = []
    no_stop  = driver.find_elements_by_class_name("stops-text")

    price =[]
    price = driver.find_elements_by_xpath("//div[@class = 'booking']")

    airlines = []
    airlines = driver.find_elements_by_xpath("//span[@class = 'codeshares-airline-names']")

    # departure_time = driver.find_elements_by_xpath("//div[@class='itinerary-depart-time depart-time']")

    price_list=[]
    for i in range(len(departure_time)):
        departure_time[i]= departure_time[i].get_attribute('innerHTML')
        arrival_time[i] = arrival_time[i].get_attribute('innerHTML')
        duration[i] = duration[i].get_attribute('innerHTML')
        no_stop[i] = no_stop[i].get_attribute('innerHTML')
        price[i] = price[i].get_attribute('innerHTML')

        airlines[i] = airlines[i].get_attribute('innerHTML')
        b = re.findall("nbsp\;((\d\d\d)|(\d.\d\d\d)|(\d\d.\d\d\d))",price[i])[0][0]
        price_list.append(b)
    
    for k in range(len(departure_time)):
        i=0
        while i<5:
            try:

                airlines[k] = airlines[k].get_attribute('innerHTML')
                i=5
            except:
                i = i+1
        
    # Creating data frames
    departure = len(departure_time)*[str(source_airport)]
    arrival = len(departure_time)*[str(destination_airport)]
    # a = pd.DataFrame(departure_time,arrival_time,price_list)
    doj1 = len(departure_time)*[doj]
    df  = pd.DataFrame(doj1,columns=["Date of journey"])
    df["Source"] = departure    
    df["Destination"] = arrival
    df["Departure_time"] = departure_time
    df["Arrival_time"] = arrival_time
    df["Duration"] = duration
    df["Stops"] = no_stop
    df["CAD $"]= price_list
    df["Airlines"] = airlines
    df["Stops"]=df["Stops"].str.replace("\n","")
    df["CAD $"]=df["CAD $"].str.replace(",","")
    df["CAD $"] = pd.to_numeric(df['CAD $'])
   
    df["Stops"]=df["Stops"].str.replace("stops","")
    df["Stops"]=df["Stops"].str.replace("stop","")
    df["Duration"]=df["Duration"].str.replace("\n","")
    df = df.drop(df[(df.Stops =='3  ') | (df.Airlines == 'Multiple Airlines')].index)
    df= df.drop_duplicates()
    df = df.sort_values(by='CAD $')
    df = df.reset_index()
    df.index = df.index +1
    df.pop("index")
    return df
    
    

In [108]:
source = ["YYZ","YYC","YYZ","YVR","YWG","YQB","YHZ","YOW"]
destination = ["DEL","BOM","MAA","BLR","COK","HYD","CCU","AMD","IXC"]


# source = ["YYZ"]
# destination = ["DEL","BOM"]
doj='2022-05-30'
df1 = pd.DataFrame(columns=['Date of journey', 'Source', 'Destination', 'Departure_time',
                            'Arrival_time', 'Duration', 'Stops', 'CAD $', 'Airlines'])
for val in source:
    for values in destination:
        source_airport = val
        destination_airport = values
        doj ='2022-05-30'
        df = kayak(doj,source_airport,destination_airport)

        df1=pd.concat([df1,df])
        



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache
<ipython-input-107-fb8e78840e62>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
<ipython-input-107-fb8e78840e62>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//a[@class='moreButton']").click()
<ipython-input-107-fb8e78840e62>:21: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  departure_time= driver.find_elements_by_xpath("//span[@class='depart-time base-time']")
<ipython-input-107-fb8e78840e62>:24: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  arrival_time

In [129]:
df2[0:10]

,Date of journey,Source,Destination,Departure_time,Arrival_time,Duration,Stops,CAD $,Airlines
1,2022-05-30,YYZ,DEL,9:40 p.m.,3:15 a.m.,20h 05m,1,933,Etihad Airways
2,2022-05-30,YYZ,DEL,2:30 p.m.,8:20 p.m.,20h 20m,1,939,Emirates
3,2022-05-30,YYZ,DEL,11:55 p.m.,8:45 a.m.,23h 20m,2,996,"Air Transat, Qatar Airways"
4,2022-05-30,YYZ,DEL,9:40 p.m.,4:40 a.m.,21h 30m,2,996,"Air Transat, Turkish Airlines"
5,2022-05-30,YYZ,DEL,12:15 p.m.,11:45 a.m.,14h 00m,direct,1115,Air India
6,2022-05-30,YYZ,DEL,10:15 p.m.,6:00 a.m.,22h 15m,1,1129,LOT
7,2022-05-30,YYZ,DEL,10:20 p.m.,4:40 a.m.,20h 50m,1,1147,Turkish Airlines
8,2022-05-30,YYZ,DEL,7:50 p.m.,12:55 a.m.,19h 35m,1,1180,Condor
9,2022-05-30,YYZ,DEL,7:50 p.m.,4:40 a.m.,23h 20m,2,1264,"Condor, Turkish Airlines"
10,2022-05-30,YYZ,DEL,9:35 p.m.,12:55 a.m.,17h 50m,1,1283,Air Canada
